In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!mkdir -p ~/.kaggle

In [3]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!pip install kaggle


In [6]:
!kaggle datasets download -d victorcallejasf/multimodal-hate-speech


Dataset URL: https://www.kaggle.com/datasets/victorcallejasf/multimodal-hate-speech
License(s): copyright-authors
100% 5.96G/5.97G [03:50<00:00, 30.3MB/s]
100% 5.97G/5.97G [03:50<00:00, 27.7MB/s]


In [7]:
!unzip /content/multimodal-hate-speech.zip -d "/content/extracted files"


Streaming output truncated to the last 5000 lines.
  inflating: /content/extracted files/img_txt/1115622578394103808.json  
  inflating: /content/extracted files/img_txt/1115622867025178625.json  
  inflating: /content/extracted files/img_txt/1115623108390653952.json  
  inflating: /content/extracted files/img_txt/1115623403040464896.json  
  inflating: /content/extracted files/img_txt/1115623527078662144.json  
  inflating: /content/extracted files/img_txt/1115623679944286208.json  
  inflating: /content/extracted files/img_txt/1115623877462327296.json  
  inflating: /content/extracted files/img_txt/1115624172321947648.json  
  inflating: /content/extracted files/img_txt/1115624287728226304.json  
  inflating: /content/extracted files/img_txt/1115624298759229440.json  
  inflating: /content/extracted files/img_txt/1115624435447468034.json  
  inflating: /content/extracted files/img_txt/1115624443341045760.json  
  inflating: /content/extracted files/img_txt/1115624455332667392.json  


In [8]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-oyg4q9un
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-oyg4q9un
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=261eb16d89613ef3ed569c52054dafaf6a0cd43c1d97d5158a0df88e77fc3d2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-wm4azv4t/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
import os
import json
import torch
import clip
from tqdm import tqdm
from PIL import Image

# Define data directories and paths
base_dir = '/content/extracted files'
img_dir = os.path.join(base_dir, 'img_resized')
text_dir = os.path.join(base_dir, 'img_txt')
labels_path = os.path.join(base_dir, 'MMHS150K_GT.json')

# Load the ground truth labels
# Load the dataset
with open(labels_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Initialize CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Define saliency points (for Gender, Disability, Race, LGBTQIA+)
gender_saliency_points = [
    "bitch", "slut", "feminazi", "stay in the kitchen", "man up", "gold digger",
    "women are property", "catfight", "misandrist", "alpha male"
]
disability_saliency_points = [
    "retard", "retarded", "cripple", "crippled", "burden on society",
    "worthless", "crazy", "spaz", "spastic", "freak", "vegetable",
    "invalid", "better off dead", "gimp"
]
race_saliency_points = [
    "nigger", "monkey", "go back to your country", "illegal", "terrorist",
    "chink", "spic", "thug", "white supremacy", "genetic inferior"
]
lgbtqia_saliency_points = [
    "faggot", "dyke", "tranny", "that's so gay", "it’s just a phase",
    "sodomite", "no homo", "groomer", "burn in hell", "agenda"
]

# Function to encode saliency points
def encode_saliency_points(points, model, device):
    text_tokens = clip.tokenize(points).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
    return text_features

# Encode the saliency points for each dimension
gender_features = encode_saliency_points(gender_saliency_points, clip_model, device)
disability_features = encode_saliency_points(disability_saliency_points, clip_model, device)
race_features = encode_saliency_points(race_saliency_points, clip_model, device)
lgbtqia_features = encode_saliency_points(lgbtqia_saliency_points, clip_model, device)

# Function to preprocess and load images
def preprocess_image(image_path):
    from PIL import Image
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0).to(device)
    return image

# Function to detect saliency points
def detect_saliency_points(text_embedding, image_embedding, dimension_features):
    saliency_scores = []
    for features in dimension_features:
        text_sim = (text_embedding @ features.T).cpu().numpy().tolist()
        image_sim = (image_embedding @ features.T).cpu().numpy().tolist()
        combined_sim = [max(t, i) for t, i in zip(text_sim[0], image_sim[0])]
        saliency_scores.append(combined_sim)
    return saliency_scores

# Function to print detected saliency points with names
def print_detected_saliency_points(detected_scores, saliency_points, dimension_name):
    detected_points = [saliency_points[i] for i, score in enumerate(detected_scores) if score > 0.3]  # Threshold can be adjusted
    print(f"{dimension_name} Dimension - Detected: {detected_points}")

# Process the dataset
def run_on_dataset(dataset, clip_model, device):
    # Loop over dataset
    for data in tqdm(dataset):  # Progress bar
        img_id = data['img_id']
        text_file = os.path.join(text_dir, f'{img_id}.txt')
        image_file = os.path.join(img_dir, f'{img_id}.jpg')

        # Load text
        with open(text_file, 'r', encoding='utf-8') as f:
            text = f.read().strip()

        # Encode text
        text_token = clip.tokenize([text]).to(device)
        with torch.no_grad():
            text_embedding = clip_model.encode_text(text_token)

        # Load and encode image
        if os.path.exists(image_file):
            image = preprocess_image(image_file)
            with torch.no_grad():
                image_embedding = clip_model.encode_image(image)
        else:
            image_embedding = text_embedding  # If no image, use text embedding

        # Detect saliency points
        saliency_results = detect_saliency_points(text_embedding, image_embedding,
                                                  [gender_features, disability_features, race_features, lgbtqia_features])

        # Print the results
        print(f"\nResults for Tweet {img_id}:")
        print_detected_saliency_points(saliency_results[0], gender_saliency_points, "Gender")
        print_detected_saliency_points(saliency_results[1], disability_saliency_points, "Disability")
        print_detected_saliency_points(saliency_results[2], race_saliency_points, "Race")
        print_detected_saliency_points(saliency_results[3], lgbtqia_saliency_points, "LGBTQIA+")

# Run the analysis on the dataset
run_on_dataset(dataset, clip_model, device)

  0%|          | 0/149823 [00:00<?, ?it/s]


TypeError: string indices must be integers

In [9]:
import os
import json
from tqdm import tqdm
import torch
import clip
from transformers import BertTokenizer
from torchvision import transforms
from PIL import Image

# Define data directories and paths
base_dir = '/content/extracted files'
img_dir = os.path.join(base_dir, 'img_resized')
text_dir = os.path.join(base_dir, 'img_txt')
labels_path = os.path.join(base_dir, 'MMHS150K_GT.json')

# Load the dataset
with open(labels_path) as f:
    label_dict = json.load(f)

# Initialize CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess_clip = clip.load("ViT-B/32", device=device)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_seq_length = 50

# Define image preprocessing function using torchvision transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit CLIP's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize pixel values
])

# Step 1: Define saliency points (as in previous steps)
gender_saliency_points = [
    "bitch", "slut", "feminazi", "stay in the kitchen", "man up", "gold digger",
    "women are property", "catfight", "misandrist", "alpha male"
]
disability_saliency_points = [
    "retard", "retarded", "cripple", "crippled", "burden on society",
    "worthless", "crazy", "spaz", "spastic", "freak", "vegetable",
    "invalid", "better off dead", "gimp"
]
race_saliency_points = [
    "nigger", "monkey", "go back to your country", "illegal", "terrorist",
    "chink", "spic", "thug", "white supremacy", "genetic inferior"
]
lgbtqia_saliency_points = [
    "faggot", "dyke", "tranny", "that's so gay", "it’s just a phase",
    "sodomite", "no homo", "groomer", "burn in hell", "agenda"
]

# Step 2: Encode saliency points using CLIP's text encoder
def encode_saliency_points(points, model, device):
    text_tokens = clip.tokenize(points).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
    return text_features

# Encode saliency points for each dimension
gender_features = encode_saliency_points(gender_saliency_points, clip_model, device)
disability_features = encode_saliency_points(disability_saliency_points, clip_model, device)
race_features = encode_saliency_points(race_saliency_points, clip_model, device)
lgbtqia_features = encode_saliency_points(lgbtqia_saliency_points, clip_model, device)

# Step 3: Function to process images
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

# Step 4: Function to detect saliency points
def detect_saliency_points(text_embedding, image_embedding, dimension_features):
    saliency_scores = []
    for features in dimension_features:
        text_sim = (text_embedding @ features.T).cpu().numpy().tolist()
        image_sim = (image_embedding @ features.T).cpu().numpy().tolist()
        combined_sim = [max(t, i) for t, i in zip(text_sim[0], image_sim[0])]
        saliency_scores.append(combined_sim)
    return saliency_scores

# Step 5: Function to print detected saliency points with names
def print_detected_saliency_points(detected_scores, saliency_points, dimension_name):
    detected_points = [saliency_points[i] for i, score in enumerate(detected_scores) if score > 0.3]  # Threshold can be adjusted
    print(f"{dimension_name} Dimension - Detected: {detected_points}")

# Step 6: Process the dataset
def run_on_dataset(label_dict, clip_model, tokenizer, device):
    for img_id, data in tqdm(label_dict.items()):  # Progress bar
        image_path = os.path.join(img_dir, img_id + '.jpg')
        text_file_path = os.path.join(text_dir, img_id + '.json')

        # Load and tokenize text
        if os.path.exists(text_file_path):
            with open(text_file_path, 'r') as f:
                text_data = json.load(f)
                text = text_data['img_text']
        else:
            text = "No Caption"

        # Tokenize the text with BERT tokenizer
        text_tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt').to(device)
        with torch.no_grad():
            text_embedding = clip_model.encode_text(clip.tokenize([text]).to(device))

        # Load and preprocess image
        if os.path.exists(image_path):
            image_embedding = preprocess_image(image_path)
            with torch.no_grad():
                image_embedding = clip_model.encode_image(image_embedding)
        else:
            image_embedding = text_embedding  # If no image is found, use text_embedding

        # Detect saliency points
        saliency_results = detect_saliency_points(text_embedding, image_embedding,
                                                  [gender_features, disability_features, race_features, lgbtqia_features])

        # Print the results
        print(f"\nResults for Tweet {img_id}:")
        print_detected_saliency_points(saliency_results[0], gender_saliency_points, "Gender")
        print_detected_saliency_points(saliency_results[1], disability_saliency_points, "Disability")
        print_detected_saliency_points(saliency_results[2], race_saliency_points, "Race")
        print_detected_saliency_points(saliency_results[3], lgbtqia_saliency_points, "LGBTQIA+")

# Step 7: Run the analysis on the dataset
run_on_dataset(label_dict, clip_model, tokenizer, device)


100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 56.3MiB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

  0%|          | 15/149823 [00:08<17:22:30,  2.39it/s]


Results for Tweet 1114918192403578884:
Gender Dimension - Detected: ['bitch', 'slut', 'feminazi', 'stay in the kitchen', 'man up', 'gold digger', 'women are property', 'catfight', 'misandrist', 'alpha male']
Disability Dimension - Detected: ['retard', 'retarded', 'cripple', 'crippled', 'burden on society', 'worthless', 'crazy', 'spaz', 'spastic', 'freak', 'vegetable', 'invalid', 'better off dead', 'gimp']
Race Dimension - Detected: ['nigger', 'monkey', 'go back to your country', 'illegal', 'terrorist', 'chink', 'spic', 'thug', 'white supremacy', 'genetic inferior']
LGBTQIA+ Dimension - Detected: ['faggot', 'dyke', 'tranny', "that's so gay", 'it’s just a phase', 'sodomite', 'no homo', 'groomer', 'burn in hell', 'agenda']


  0%|          | 16/149823 [00:08<22:29:52,  1.85it/s]


Results for Tweet 1116392705716830208:
Gender Dimension - Detected: ['bitch', 'slut', 'feminazi', 'stay in the kitchen', 'man up', 'gold digger', 'women are property', 'catfight', 'misandrist', 'alpha male']
Disability Dimension - Detected: ['retard', 'retarded', 'cripple', 'crippled', 'burden on society', 'worthless', 'crazy', 'spaz', 'spastic', 'freak', 'vegetable', 'invalid', 'better off dead', 'gimp']
Race Dimension - Detected: ['nigger', 'monkey', 'go back to your country', 'illegal', 'terrorist', 'chink', 'spic', 'thug', 'white supremacy', 'genetic inferior']
LGBTQIA+ Dimension - Detected: ['faggot', 'dyke', 'tranny', "that's so gay", 'it’s just a phase', 'sodomite', 'no homo', 'groomer', 'burn in hell', 'agenda']


RuntimeError: Input Sierra Club a partagé une vidéo en direct. Eisa Today is the United Nation's "International Day of the Girl" and we want to celebrate the awesome girls of Girl Scout Troop 6195 in llinois These young women are powerful advocates for clean air and clean water, having joined the local Sierra Club group at numerous events to speak up for the environment and climate action. Most recently they testified in favor of protecting the historic Clean Power Plan, a climate action plan that's at risk of being dismantied. Watch their speech at the rally outside the hearing!  is too long for context length 77

In [10]:
import os
import json
from tqdm import tqdm
import torch
import clip
from transformers import BertTokenizer
from torchvision import transforms
from PIL import Image

# Define data directories and paths
base_dir = '/content/extracted files'
img_dir = os.path.join(base_dir, 'img_resized')
text_dir = os.path.join(base_dir, 'img_txt')
labels_path = os.path.join(base_dir, 'MMHS150K_GT.json')

# Load the dataset
with open(labels_path) as f:
    label_dict = json.load(f)

# Initialize CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess_clip = clip.load("ViT-B/32", device=device)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_seq_length = 50

# Define image preprocessing function using torchvision transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit CLIP's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize pixel values
])

# Step 1: Define saliency points (as in previous steps)
gender_saliency_points = [
    "bitch", "slut", "feminazi", "stay in the kitchen", "man up", "gold digger",
    "women are property", "catfight", "misandrist", "alpha male"
]
disability_saliency_points = [
    "retard", "retarded", "cripple", "crippled", "burden on society",
    "worthless", "crazy", "spaz", "spastic", "freak", "vegetable",
    "invalid", "better off dead", "gimp"
]
race_saliency_points = [
    "nigger", "monkey", "go back to your country", "illegal", "terrorist",
    "chink", "spic", "thug", "white supremacy", "genetic inferior"
]
lgbtqia_saliency_points = [
    "faggot", "dyke", "tranny", "that's so gay", "it’s just a phase",
    "sodomite", "no homo", "groomer", "burn in hell", "agenda"
]

# Step 2: Encode saliency points using CLIP's text encoder
def encode_saliency_points(points, model, device):
    text_tokens = clip.tokenize(points).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
    return text_features

# Encode saliency points for each dimension
gender_features = encode_saliency_points(gender_saliency_points, clip_model, device)
disability_features = encode_saliency_points(disability_saliency_points, clip_model, device)
race_features = encode_saliency_points(race_saliency_points, clip_model, device)
lgbtqia_features = encode_saliency_points(lgbtqia_saliency_points, clip_model, device)

# Step 3: Function to process images
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

# Step 4: Function to detect saliency points
def detect_saliency_points(text_embedding, image_embedding, dimension_features):
    saliency_scores = []
    for features in dimension_features:
        text_sim = (text_embedding @ features.T).cpu().numpy().tolist()
        image_sim = (image_embedding @ features.T).cpu().numpy().tolist()
        combined_sim = [max(t, i) for t, i in zip(text_sim[0], image_sim[0])]
        saliency_scores.append(combined_sim)
    return saliency_scores

# Step 5: Function to print detected saliency points with names
def print_detected_saliency_points(detected_scores, saliency_points, dimension_name):
    detected_points = [saliency_points[i] for i, score in enumerate(detected_scores) if score > 0.3]  # Threshold can be adjusted
    print(f"{dimension_name} Dimension - Detected: {detected_points}")

# Step 6: Process the dataset
def run_on_dataset(label_dict, clip_model, tokenizer, device):
    for img_id, data in tqdm(label_dict.items()):  # Progress bar
        image_path = os.path.join(img_dir, img_id + '.jpg')
        text_file_path = os.path.join(text_dir, img_id + '.json')

        # Load and tokenize text
        if os.path.exists(text_file_path):
            with open(text_file_path, 'r') as f:
                text_data = json.load(f)
                text = text_data['img_text']
        else:
            text = "No Caption"

        # Truncate the text to the first 77 tokens if necessary
        tokenized_clip_text = clip.tokenize(text[:77]).to(device)

        with torch.no_grad():
            text_embedding = clip_model.encode_text(tokenized_clip_text)

        # Load and preprocess image
        if os.path.exists(image_path):
            image_embedding = preprocess_image(image_path)
            with torch.no_grad():
                image_embedding = clip_model.encode_image(image_embedding)
        else:
            image_embedding = text_embedding  # If no image is found, use text_embedding

        # Detect saliency points
        saliency_results = detect_saliency_points(text_embedding, image_embedding,
                                                  [gender_features, disability_features, race_features, lgbtqia_features])

        # Print the results
        print(f"\nResults for Tweet {img_id}:")
        print_detected_saliency_points(saliency_results[0], gender_saliency_points, "Gender")
        print_detected_saliency_points(saliency_results[1], disability_saliency_points, "Disability")
        print_detected_saliency_points(saliency_results[2], race_saliency_points, "Race")
        print_detected_saliency_points(saliency_results[3], lgbtqia_saliency_points, "LGBTQIA+")

# Step 7: Run the analysis on the dataset
run_on_dataset(label_dict, clip_model, tokenizer, device)


  0%|          | 1/149823 [00:00<16:06:15,  2.58it/s]


Results for Tweet 1114679353714016256:
Gender Dimension - Detected: ['bitch', 'slut', 'feminazi', 'stay in the kitchen', 'man up', 'gold digger', 'women are property', 'catfight', 'misandrist', 'alpha male']
Disability Dimension - Detected: ['retard', 'retarded', 'cripple', 'crippled', 'burden on society', 'worthless', 'crazy', 'spaz', 'spastic', 'freak', 'vegetable', 'invalid', 'better off dead', 'gimp']
Race Dimension - Detected: ['nigger', 'monkey', 'go back to your country', 'illegal', 'terrorist', 'chink', 'spic', 'thug', 'white supremacy', 'genetic inferior']
LGBTQIA+ Dimension - Detected: ['faggot', 'dyke', 'tranny', "that's so gay", 'it’s just a phase', 'sodomite', 'no homo', 'groomer', 'burn in hell', 'agenda']


  0%|          | 2/149823 [00:00<16:05:31,  2.59it/s]


Results for Tweet 1063020048816660480:
Gender Dimension - Detected: ['bitch', 'slut', 'feminazi', 'stay in the kitchen', 'man up', 'gold digger', 'women are property', 'catfight', 'misandrist', 'alpha male']
Disability Dimension - Detected: ['retard', 'retarded', 'cripple', 'crippled', 'burden on society', 'worthless', 'crazy', 'spaz', 'spastic', 'freak', 'vegetable', 'invalid', 'better off dead', 'gimp']
Race Dimension - Detected: ['nigger', 'monkey', 'go back to your country', 'illegal', 'terrorist', 'chink', 'spic', 'thug', 'white supremacy', 'genetic inferior']
LGBTQIA+ Dimension - Detected: ['faggot', 'dyke', 'tranny', "that's so gay", 'it’s just a phase', 'sodomite', 'no homo', 'groomer', 'burn in hell', 'agenda']


  0%|          | 3/149823 [00:01<16:28:20,  2.53it/s]

...

  1%|          | 1163/149823 [10:28<42:58:40,  1.04s/it]


Results for Tweet 1036290863742242816:
Gender Dimension - Detected: ['bitch', 'slut', 'feminazi', 'stay in the kitchen', 'man up', 'gold digger', 'women are property', 'catfight', 'misandrist', 'alpha male']
Disability Dimension - Detected: ['retard', 'retarded', 'cripple', 'crippled', 'burden on society', 'worthless', 'crazy', 'spaz', 'spastic', 'freak', 'vegetable', 'invalid', 'better off dead', 'gimp']
Race Dimension - Detected: ['nigger', 'monkey', 'go back to your country', 'illegal', 'terrorist', 'chink', 'spic', 'thug', 'white supremacy', 'genetic inferior']
LGBTQIA+ Dimension - Detected: ['faggot', 'dyke', 'tranny', "that's so gay", 'it’s just a phase', 'sodomite', 'no homo', 'groomer', 'burn in hell', 'agenda']


  1%|          | 1164/149823 [10:29<22:19:11,  1.85it/s]


Results for Tweet 1108752448016076800:
Gender Dimension - Detected: ['bitch', 'slut', 'feminazi', 'stay in the kitchen', 'man up', 'gold digger', 'women are property', 'catfight', 'misandrist', 'alpha male']
Disability Dimension - Detected: ['retard', 'retarded', 'cripple', 'crippled', 'burden on society', 'worthless', 'crazy', 'spaz', 'spastic', 'freak', 'vegetable', 'invalid', 'better off dead', 'gimp']
Race Dimension - Detected: ['nigger', 'monkey', 'go back to your country', 'illegal', 'terrorist', 'chink', 'spic', 'thug', 'white supremacy', 'genetic inferior']
LGBTQIA+ Dimension - Detected: ['faggot', 'dyke', 'tranny', "that's so gay", 'it’s just a phase', 'sodomite', 'no homo', 'groomer', 'burn in hell', 'agenda']


RuntimeError: Input g98、腾讯微博 http://t gq.cormenena 金沙江大桥垮了!最让官方伤心的是,这桥太不争 气,居然在没车的时候跨!这还了得,以前桥垮,都 is too long for context length 77